<a href="https://colab.research.google.com/github/SY-256/anomaly_detection/blob/main/chapter3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVMによる異常検知の実装

In [ ]:
from sklearn.svm import SVC
from sklearn.inspection import DecisionBoundaryDisplay
import matplotlib.cm as cm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 学習

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/ch2_dataset_train.csv")

In [ ]:
from math import gamma
# "temp1", "temp2"変数に欠損があるデータを削除
df_dropna = df.dropna(subset=["temp1", "temp2"])
# データをnumpy.ndarrayに変換
X = df_dropna[["temp2", "temp1"]].to_numpy()
y = df_dropna["label"].to_numpy()

# SVMモデル
svm = SVC(C=10, gamma=0.1)
# 標準化とSVMを合わせてパイプライン化する
clf = Pipeline([("scaler", StandardScaler()), ("svm", svm)])
clf.fit(X, y)
# 決定境界をプロット
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
disp = DecisionBoundaryDisplay.from_estimator(
    clf, X, response_method="predict",
    xlabel="temp2", ylabel="temp1",
    cmap=cm.gray, alpha=0.5, ax=ax
)
# 散布図を重ねてプロット
sns.scatterplot(
    data=df_dropna, x="temp2", y="temp1",
    hue="label", palette=["#999999", "#111111"],
    ax=ax
)
plt.show()

- SVMは欠損のあるデータには適用できない
- SVMは特徴空間状での距離（ユークリッド距離）に基づくアルゴリズムのため、前処理として標準化を実施する必要がある

## 推論

In [ ]:
# 推論データ読み込み
df_inference = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/ch2_dataset_inference.csv")
# "temp2", "temp1"で欠損があるデータを削除
df_inference_dropna = df_inference.dropna(subset=["temp2", "temp1"])
# numpy.ndarrayに変換
X_inference = df_inference_dropna[["temp2", "temp1"]].to_numpy()
y_inference = df_inference_dropna["label"].to_numpy()
# 学習済みのSVMモデルで推論
y_pred = clf.predict(X_inference)
# 推論結果を表示
print(y_pred)

In [ ]:
# モデルの保存
import pickle
filepath = "./svm_model.pkl"
with open(filepath, "wb") as f:
    pickle.dump(clf, f)

In [ ]:
# モデルのロード
filepath = "./svm_model.pkl"
with open(filepath, "rb") as f:
    clf = pickle.load(f)

## 過学習とハイパーパラメータチューニング

In [ ]:
# ガウスカーネルの分散の逆数パラメータgammaを変化させた際の決定境界をプロット
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))
for i, gamma in enumerate([10, 1, 0.1, 0.01]):
    # SVMを学習
    svm = SVC(C=10, gamma=gamma)
    clf = Pipeline([("scaler", StandardScaler()), ("svm", svm)])
    clf.fit(X, y)
    # 決定境界をプロット
    disp = DecisionBoundaryDisplay.from_estimator(
        clf, X, response_method="predict",
        xlabel="temp2", ylabel="temp1",
        cmap=cm.gray, alpha=0.5, ax=ax[i]
    )
    # 散布図を重ねてプロット
    sns.scatterplot(
        data=df_dropna, x="temp2", y="temp1",
        hue="label", palette = ["#999999", "#111111"],
        ax=ax[i]
    )
    ax[i].set_title(f"gamma={gamma}", fontsize=16)
plt.show()

In [ ]:
# 正則化項パラメータCの値を変えて決定境界をプロット
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))
for i, C in enumerate([1000, 100, 10, 1]):
    # SVMを学習
    svm = SVC(C=C, gamma=0.1)
    clf = Pipeline([("scaler", StandardScaler()), ("svm", svm)])
    clf.fit(X, y)
    # 決定境界をプロット
    disp = DecisionBoundaryDisplay.from_estimator(
        clf, X, response_method="predict",
        xlabel="temp2", ylabel="temp1",
        cmap=cm.gray, alpha=0.5, ax=ax[i]
    )
    # 散布図を重ねてプロット
    sns.scatterplot(
        data=df_dropna, x="temp2", y="temp1",
        hue="label", palette=["#999999", "#111111"],
        ax=ax[i]
    )
    ax[i].set_title(f"C={C}", fontsize=16)
plt.show()

- Cが大きいと正則化項の効果が小さくなるため、過学習寄りになる->決定境界がいびつな形になる